In [1]:
# Installed packages
import os
import numpy as np
import scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, MaxNLocator)
import datetime
import matplotlib.dates as mdates

# Local files
import CTP_fns

In [ ]:
# Load the data from the uploaded files. Files were given in .xlsx and have been converted to .csv
input_dir = "input_data"
input_data = {
                fn[:-4]: pd.read_csv(f"{input_dir}/{fn}", index_col=False) for fn in os.listdir(input_dir)
            }
# Identify the column with datetime for analysis
time_col = {
                "PatEnt": "createdAt_time",
                "AuAx": "timeResponded",
            }

# Identify and remove problematic rows where the data contains the invalid datetime: 29/02/2014
for fn in input_data:
    input_data[fn] = input_data[fn].iloc[~CTP_fns.invalid_leapday(input_data[fn][time_col[fn]].values)]
    #input_data[fn][time_col[fn]] = pd.to_datetime(input_data[fn][time_col[fn]])

# Display input data
input_data

In [ ]:
# Display the cleaned Patients Entries DataFrame
input_data["PatEnt"]

In [ ]:
# Display the cleaned AuAx DataFrame
input_data["AuAx"]

In [ ]:
# Identify and display unique patientId values in Patient Entries
unique_patientIds = np.unique(input_data["Patient Entries"]["patientId"])
unique_patientIds

In [6]:
# Create dictionary for output DataFrames
output_data = {}

In [ ]:
# Create DataFrame for output
output_df = pd.DataFrame()

# Create null Audit Actions data slice for when a entry in Patient Entries has no matching response in Audit Actions
null_AA_slice = {
                    "action": "",
                    "userId": "",
                    "teamId": "",
                    "timeResponded": "1970-01-01 00:00:00.000000 UTC",    
                    "organisationId": "",           
                }


for pid in unique_patientIds:

    # Slice Patient Entries and Audit Actions w.r.t. the unique patientId (pid)
    PE = CTP_fns.index_df(input_data["Patient Entries"], colval={"patientId": [pid]}).sort_values("createdAt_time")
    AA = CTP_fns.index_df(input_data["Audit Actions"], colval={"patientId": [pid]}).sort_values("timeResponded")

    # Extract arrays of unique createdAt_time values in PE and timeResponded values in AA
    timeCreated = np.unique(PE["createdAt_time"])
    timeResponded = AA.sort_values("timeResponded")["timeResponded"].values

    # Convert to datetime
    timeC = pd.to_datetime(timeCreated)
    timeR = pd.to_datetime(timeResponded)

    # Match PE events (rows in PE grouped by same createdAt_time) to AA events (rows in AA)
    # Matches made within each pid slice by minimum positive time difference between PE event timeC and AA event timeR
    response_index = [None for i in range(len(timeC))]
    if len(timeResponded)>0:
        # Create matrix of timedeltas between timeR and timeC
        deltas = np.add.outer(timeR - timeC[0], -(timeC - timeC[0]))/pd.Timedelta(seconds=1)
        if len(timeR)>=len(timeC):  
            CtR = np.argmin(np.ma.MaskedArray(deltas, deltas<0), axis=0)
            for k in range(len(timeC)):
                if deltas[CtR[k], k]>0:
                    response_index[k] = CtR[k]
        else:
            RtC = np.argmin(np.ma.MaskedArray(deltas, deltas<0), axis=1)
            for k in range(len(RtC)):
                if deltas[k, RtC[k]]>0:
                    response_index[RtC[k]] = k
        

    for i in range(len(timeCreated)):
        # Extract first row of PE event
        PE_slice = CTP_fns.index_df(input_data["Patient Entries"], 
                        colval={
                            "patientId": [pid],
                            "createdAt_time": [timeCreated[i]],                  
                            },
                        ).iloc[0]
        
        responded = response_index[i] is not None
        if responded:
            # Extract matching AA event
            AA_slice = CTP_fns.index_df(input_data["Audit Actions"], 
                            colval={
                                "patientId": [pid],
                                "timeResponded": [timeResponded[response_index[i]]],                  
                                },
                            ).iloc[0]
        else:
            # Use null for missing AA event
            AA_slice = null_AA_slice

        # Add composite row to output_df DataFrame
        output_df = pd.concat([output_df, pd.DataFrame({
            **PE_slice,
            **AA_slice,
            "responded": responded,
            }, index=[0])], ignore_index=True)

# Convert timeResponded and createdAt_time columns to datetime
for col in ["timeResponded", "createdAt_time"]:
    output_df[col] = pd.to_datetime(output_df[col])

# Add column identifying month_year of PE events
output_df["month_year"] = output_df["createdAt_time"].dt.to_period('M')

# Add column for timedelta between PE event createdAt_time and AA event timeResponded
output_df["response_time"] = output_df["timeResponded"] - output_df["createdAt_time"]

# Save output_df in output_data as "Responses"
output_data["Responses"] = output_df

In [ ]:
# Display the Responses DataFrame
output_data["Responses"]

In [ ]:
CTP_fns.index_df(output_data["Responses"], colval={"team name": "Team D",
                                                   "responded": True})

In [ ]:
pid = "vGvpDeNAYT"

CTP_fns.index_df(output_data["Responses"], colval={"patientId": pid})

In [ ]:
pid = "vGvpDeNAYT"

CTP_fns.index_df(input_data["Patient Entries"], colval={"patientId": pid})

In [ ]:
CTP_fns.index_df(input_data["Patient Entries"], colval={"patientId": pid}).sort_values("createdAt_time").iloc[40:]

In [ ]:
pid = "vGvpDeNAYT"

CTP_fns.index_df(input_data["Audit Actions"], colval={"patientId": pid})

In [ ]:
CTP_fns.index_df(output_data["Responses"], colval={"team name": "Team B",
                                                   "responded": True})

In [ ]:
pid = "p5XG3CyQok"

CTP_fns.index_df(output_data["Responses"], colval={"patientId": pid})

In [ ]:
pid = "7Wbihwn5Ly"

CTP_fns.index_df(input_data["Patient Entries"], colval={"patientId": pid})

In [ ]:
pid = "ZBw6xgfs8w"

CTP_fns.index_df(input_data["Audit Actions"], colval={"patientId": pid})

In [ ]:
pid = "7Wbihwn5Ly"

CTP_fns.index_df(output_data["Responses"], colval={"patientId": pid})

In [ ]:
pid = "vTkccePogn"

CTP_fns.index_df(input_data["Patient Entries"], colval={"patientId": pid})

In [ ]:
pid = "vTkccePogn"

CTP_fns.index_df(input_data["Audit Actions"], colval={"patientId": pid})

In [21]:
# Create DataFrame for output
output_df = pd.DataFrame()

# Set input_df as "Responses"
input_df = output_data["Responses"]

# Identify unique team name values in input_df
unique_team_names = np.unique(input_df["team name"])

for tid in unique_team_names:
    # Slice input_df by unique team name (tid) and extract response_time column for rows where responded=True
    slice_cv = {"team name": tid}
    response_times = CTP_fns.index_df(input_df, colval={"responded": True, **slice_cv})["response_time"]
    
    # Create new evaluation row from response_times
    team_eval = pd.DataFrame({
                                "team_name": tid,
                                "N_events": len(response_times),
                                "response_fraction": np.mean(CTP_fns.index_df(input_df, colval=slice_cv)["responded"]),
                                "response_time_mean": CTP_fns.timedelta_mean(response_times),
                                "response_time_median": CTP_fns.timedelta_quantile(response_times, q=0.5),
                                "response_time_std": CTP_fns.timedelta_std(response_times),
                            },
                            index=[0]
                            )
    
    # Add new evaluation row to output_df DataFrame
    output_df = pd.concat([output_df, team_eval], ignore_index=True)

# Convert selected columns to timedelta dtype
for col in output_df.columns:
    if col[:13]=="response_time":
        output_df[col] = pd.to_timedelta(output_df[col])

# Save output_df in output_data as "team_eval"
output_data["team_eval"] = output_df

In [ ]:
# Display the team_eval DataFrame
output_data["team_eval"]

In [23]:
# Create DataFrame for output
output_df = pd.DataFrame()

# Set input_df as "Responses"
input_df = output_data["Responses"]

# Identify unique month_year values in input_df
unique_month_year = np.unique(CTP_fns.index_df(input_df)["month_year"])

for my in unique_month_year:
    # Slice input_df by unique month_year (my) and extract response_time column for rows where responded=True
    slice_cv = {"month_year": my}
    response_times = CTP_fns.index_df(input_df, colval={"responded": True, **slice_cv})["response_time"]

    # Create new evaluation row from response_times
    my_eval = pd.DataFrame({
                                "month_year": my,
                                "N_events": len(response_times),
                                "response_fraction": np.mean(CTP_fns.index_df(output_data["Responses"], colval=slice_cv)["responded"]),
                                "response_time_mean": CTP_fns.timedelta_mean(response_times),
                                "response_time_median": CTP_fns.timedelta_quantile(response_times, q=0.5),
                                "response_time_std": CTP_fns.timedelta_std(response_times),
                            },
                            index=[0]
                            )
    
    # Add new evaluation row to output_df DataFrame
    output_df = pd.concat([output_df, my_eval], ignore_index=True)

# Convert selected columns to timedelta dtype
for col in output_df.columns:
    if col[:13]=="response_time":
        output_df[col] = pd.to_timedelta(output_df[col])

# Save output_df in output_data as "org_eval"
output_data["month_eval"] = output_df

In [ ]:
# Display the month_eval DataFrame
output_data["month_eval"]

In [25]:
# Create DataFrame for output
output_df = pd.DataFrame()

# Set input_df as "Responses"
input_df = output_data["Responses"]


for tid in unique_team_names:

    # Identify unique month_year values in input_df sliced by unique team name (tid)
    tid_unique_month_year = np.unique(CTP_fns.index_df(output_data["Responses"], colval={"team name": tid})["month_year"])

    for my in tid_unique_month_year:
        # Slice input_df by unique team name (tid) month_year (my) and extract response_time column for rows where responded=True
        slice_cv = {"team name": tid, "month_year": my}
        response_times = CTP_fns.index_df(input_df, colval={"responded": True, **slice_cv})["response_time"]
            
        team_month_eval = pd.DataFrame({
                                        "team_name": tid,
                                        "month_year": my,
                                        "N_events": len(response_times),
                                        "response_fraction": np.mean(CTP_fns.index_df(input_df, colval=slice_cv)["responded"]),
                                        "response_time_mean": CTP_fns.timedelta_mean(response_times),
                                        "response_time_median": CTP_fns.timedelta_quantile(response_times, q=0.5),
                                        "response_time_std": CTP_fns.timedelta_std(response_times),
                                    },
                                    index=[0]
                                    )
            
        output_df = pd.concat([output_df, team_month_eval], ignore_index=True)


# Convert selected columns to timedelta dtype
for col in output_df.columns:
    if col[:13]=="response_time":
        output_df[col] = pd.to_timedelta(output_df[col])

# Save output_df in output_data as "org_eval"
output_data["team_month_eval"] = output_df

In [ ]:
# Display the team_month_eval DataFrame
output_data["team_month_eval"]

In [27]:
# Create DataFrame for output
output_df = pd.DataFrame()

# Set input_df as "team_month_eval"
input_df = output_data["team_month_eval"]

# Identify unique month_year values in input_df
unique_month_year = np.unique(input_df["month_year"])

for my in unique_month_year:

    # Extract month_eval by slicing input_df by unique month_year (my) and sorting rows by "response_time_mean"
    slice_cv = {"month_year": my}
    month_eval = CTP_fns.index_df(input_df, colval=slice_cv).sort_values("response_time_mean")
    # Select best team (bt) from top row (i.e. shortest response_time_mean)
    bt = month_eval["team_name"].values[0]

    # Extract best team row from month_eval and append to output_df
    bt_month_eval = CTP_fns.index_df(month_eval, colval={"team_name": bt})
    output_df = pd.concat([output_df, bt_month_eval], ignore_index=True)

# Save output_df in output_data as "bt_month_eval"
output_data["bt_month_eval"] = output_df

In [ ]:
# Display the bt_month_eval DataFrame
output_data["bt_month_eval"]

In [ ]:
# Select column to be plotted
val = "response_time_mean"
val2 = "N_events"
# Create figure object
fig_team = CTP_fns.rc_fig(name=f"fig_team_{val}")

# Set plot_df as "team_eval" sorted by "team_name"
plot_df = output_data["team_eval"].sort_values("team_name")

# Plot ax
x = np.arange(plot_df.shape[0]) # Placement of bars along x
y = (plot_df[val]/pd.Timedelta(hours=1)).values # Bar heights in units of hours
fig_team.ax[0][0].bar(x=x, height=y) # Plot bars
fig_team.ax[0][0].set_xlim(x[[0,-1]] + 0.5*np.array([-1,1]))

# Plot ax2
fig_team.twin("x")
y2 = plot_df[val2].values/len(unique_month_year) # Values on y on ax2
fig_team.ax2[0][0].plot(x, y2, marker=".", color="black", lw=2, ms=10, ls=":") # Plot line
fig_team.ax2[0][0].set_ylim(bottom=0)

# Legend
fig_team.ax[0][0].legend([fig_team.__dict__[f"ax{k}"][0][0].get_children()[0] for k in ["", "2"]], [val, val2], loc="upper left") # Legend

# team_name ticks
xticklabels_options = {"rotation": 45, "horizontalalignment": 'right', "rotation_mode": 'anchor'} 
fig_team.ax[0][0].set_xticks(x, labels=np.array(plot_df["team_name"].values, dtype=str), **xticklabels_options)
# ylabels
fig_team.ax[0][0].set_ylabel(f"{val} / hr")
fig_team.ax2[0][0].set_ylabel(f"{val2} / month")
fig_team.ax[0][0].set_title("Performance by Team")

# Export figure as pdf
fig_team.export(fig_dir="output_figs")


In [ ]:
# Select column to be plotted
val = "response_time_mean"
val2 = "N_events"
# Create figure object
fig_month = CTP_fns.rc_fig(name=f"fig_month_{val}")

# Set plot_df as "team_eval" sorted by "team_name"
plot_df = output_data["month_eval"].sort_values("month_year")

# Plot ax
x = np.array([month.strftime('%b %Y') for month in plot_df["month_year"].values])  # Timestamps on x
y = (plot_df[val]/pd.Timedelta(hours=1)).values # Values on y in units of hours
fig_month.ax[0][0].plot(x, y, marker=".", lw=2, ms=10)    # Plot line

# Plot ax2
fig_month.twin("x")
y2 = plot_df[val2].values # Values on y on ax2
fig_month.ax2[0][0].plot(x, y2, marker=".", color="black", lw=2, ms=10) # Plot line
#fig_month.ax2[0][0].set_ylim(bottom=0)

# Legend
fig_month.ax[0][0].legend([fig_month.__dict__[f"ax{k}"][0][0].get_children()[0] for k in ["", "2"]], [val, val2], loc="upper center") # Legend

# month ticks
xticklabels_options = {"rotation": 45, "horizontalalignment": 'right', "rotation_mode": 'anchor'} 
fig_month.ax[0][0].set_xticks(fig_month.ax[0][0].get_xticks(), labels=fig_month.ax[0][0].get_xticklabels(), **xticklabels_options)
# ylabels
fig_month.ax[0][0].set_ylabel(f"{val} / hr")
fig_month.ax2[0][0].set_ylabel(f"{val2}")
fig_month.ax2[0][0].yaxis.set_major_locator(MaxNLocator(integer=True))
fig_month.ax[0][0].set_title("Overall Performance by Month")

# Export figure as pdf
fig_month.export(fig_dir="output_figs")

In [ ]:
# Select column to be plotted
val = "response_time_mean"
val2 = "N_events"
# Create figure object
fig_bt_month = CTP_fns.rc_fig(name=f"fig_bt_month_{val}")

# Set plot_df as "team_eval" sorted by "team_name"
plot_df = output_data["bt_month_eval"].sort_values("month_year")

# Colours for team_name
cols = CTP_fns.make_iterable_array(plt.rcParams['axes.prop_cycle'].by_key()['color'])  # Colour prop_cycle
unique_bt_names = np.unique(plot_df["team_name"])   # Unique best team_names (btid)
btid_Series = pd.Series(np.arange(unique_bt_names.size), unique_bt_names) # Colour index for btid values
bt_cols = cols[btid_Series[plot_df["team_name"]]] # Sequential bar colours
btid_i0 = pd.Series(np.array([plot_df.index[CTP_fns.get_index(plot_df, colval={"team_name": btid})][0] for btid in unique_bt_names]), index=unique_bt_names) # Index of first bar for each btid

# Plot ax
x = np.array([month.strftime('%b %Y') for month in plot_df["month_year"].values])  # Timestamps on x
y = (plot_df[val]/pd.Timedelta(hours=1)).values # Values on y in units of hours
fig_bt_month.ax[0][0].bar(x=x, height=y, width=0.8, color=bt_cols) # Plot bars

# Plot ax2
fig_bt_month.twin("x")
y2 = plot_df[val2].values # Values on y on ax2
fig_bt_month.ax2[0][0].plot(x, y2, marker=".", color="black", lw=2, ms=10, ls=":") # Plot line
fig_bt_month.ax2[0][0].set_ylim(bottom=0)

# Legends
fig_bt_month.ax[0][0].legend([fig_bt_month.ax[0][0].get_children()[k] for k in btid_i0.values], btid_i0.index) # ax
fig_bt_month.ax2[0][0].legend([fig_bt_month.ax2[0][0].get_children()[k] for k in [0]], [val2], loc="upper center") # ax2

# month ticks
xticklabels_options = {"rotation": 45, "horizontalalignment": 'right', "rotation_mode": 'anchor'} 
fig_bt_month.ax[0][0].set_xticks(fig_bt_month.ax[0][0].get_xticks(), labels=fig_bt_month.ax[0][0].get_xticklabels(), **xticklabels_options)
# ylabels
fig_bt_month.ax[0][0].set_ylabel(f"{val} / hr")
fig_bt_month.ax2[0][0].set_ylabel(f"{val2}")
fig_bt_month.ax2[0][0].yaxis.set_major_locator(MaxNLocator(integer=True))
fig_bt_month.ax[0][0].set_title("Best Team Performance by Month")

# Export figure as pdf
fig_bt_month.export(fig_dir="output_figs")

In [32]:
# Export output_data to .csv files in output_dir
output_dir = "output_data"
CTP_fns.mkdir_export(output_dir)
for fn in output_data:
    output_data[fn].to_csv(f"{output_dir}/{fn}.csv", index=False)